# Routing Demo — API Explorer
Make sure the backend is running before executing these cells:
```
cd backend && .venv/bin/uvicorn main:app --reload
```

In [1]:
import json
import requests

BASE_URL = "http://localhost:8000"

## Available nodes
Fetch all candidate locations — use these lat/lon values when building requests.

In [2]:
nodes = requests.get(f"{BASE_URL}/nodes").json()
for n in nodes:
    print(f"  [{n['id']:2d}] {n['name']:<20} ({n['lat']}, {n['lon']})")

  [ 0] McCordsville         (39.87615, -85.920833)
  [ 1] Lawernencburg        (39.109749, -84.843031)
  [ 2] Dayton               (39.744506, -84.190569)
  [ 3] Louisville           (38.226329, -85.69371)
  [ 4] Indianapolis         (39.747681, -86.174798)
  [ 5] Cincinnati           (39.100085, -84.513169)
  [ 6] Colombus             (39.926166, -83.004087)
  [ 7] Oxford               (39.486617, -84.74985)
  [ 8] Greenville           (40.06944, -84.632968)
  [ 9] Chilicothe           (39.320697, -82.990459)
  [10] Connersville         (39.621281, -85.14425)
  [11] Hillsboro            (39.178211, -83.615409)
  [12] Bloomington          (39.150817, -86.543169)
  [13] Batesville           (39.282208, -85.219435)
  [14] Williamstown         (38.622799, -84.557569)
  [15] Urbana               (40.092413, -83.742601)
  [16] Eaton                (39.714776, -84.59761)
  [17] Jefferson            (39.63138, -83.549458)
  [18] Westport             (39.15995, -85.582167)
  [19] Piqua        

## Submit an optimization request
Edit the payload below to experiment with different sources, destinations, containers, and truck sizes.

In [3]:
with open("../data/test_data.json") as f:
    payload = json.load(f)

print(json.dumps(payload, indent=2))

{
  "sources": [
    {
      "id": "src-0",
      "lat": "39.876150",
      "lon": "-85.920833"
    },
    {
      "id": "src-1",
      "lat": "39.747681",
      "lon": "-86.174798"
    }
  ],
  "destinations": [
    {
      "id": "dst-0",
      "lat": "39.109749",
      "lon": "-84.843031"
    },
    {
      "id": "dst-1",
      "lat": "39.744506",
      "lon": "-84.190569"
    },
    {
      "id": "dst-2",
      "lat": "38.226329",
      "lon": "-85.693710"
    },
    {
      "id": "dst-3",
      "lat": "39.100085",
      "lon": "-84.513169"
    },
    {
      "id": "dst-4",
      "lat": "39.926166",
      "lon": "-83.004087"
    }
  ],
  "containers": [
    {
      "container_id": "c-00",
      "source_id": "src-0",
      "destination_id": "dst-0",
      "size": 3,
      "temperature": "AM"
    },
    {
      "container_id": "c-01",
      "source_id": "src-0",
      "destination_id": "dst-0",
      "size": 2,
      "temperature": "RE"
    },
    {
      "container_id": "c-02",
     

In [4]:
response = requests.post(f"{BASE_URL}/optimize", json=payload)
response.raise_for_status()
result = response.json()
print(json.dumps(result, indent=2))

{
  "trucks": [
    {
      "id": "403dce9a-0f15-48bb-861c-4607039eaaa0",
      "source_id": "src-0",
      "destination_ids": [
        "dst-0",
        "dst-3",
        "dst-1"
      ],
      "container_ids": [
        "c-00",
        "c-01",
        "c-02",
        "c-03",
        "c-05",
        "c-06"
      ],
      "route_distance_meters": 289202,
      "route_duration_seconds": 11436
    },
    {
      "id": "580694d9-f30c-4b5a-995f-2f5843f24817",
      "source_id": "src-0",
      "destination_ids": [
        "dst-2"
      ],
      "container_ids": [
        "c-04"
      ],
      "route_distance_meters": 212385,
      "route_duration_seconds": 7565
    },
    {
      "id": "f4d3ce41-21ee-48ce-a945-a0edbab78aa9",
      "source_id": "src-1",
      "destination_ids": [
        "dst-3",
        "dst-1",
        "dst-4",
        "dst-2"
      ],
      "container_ids": [
        "c-07",
        "c-08",
        "c-09",
        "c-10",
        "c-11"
      ],
      "route_distance_meter

## Summarise results

In [8]:
node_id_to_name = {n["id"]: n["name"] for n in nodes}

# Build a lookup from the original payload for container and location details
container_map = {c["container_id"]: c for c in payload["containers"]}
source_map    = {s["id"]: s for s in payload["sources"]}
dest_map      = {d["id"]: d for d in payload["destinations"]}

print(f"{len(result['trucks'])} truck(s) dispatched\n")
for truck in result["trucks"]:
    src_id = truck["source_id"]
    dest_node_ids = truck["destination_ids"]
    containers = [container_map[cid] for cid in truck["container_ids"]]

    print(f"Truck {truck['id'][:8]}...")
    print(f"  Source     : {src_id}")
    print(f"  Route      : {' -> '.join(node_id_to_name.get(d, str(d)) for d in dest_node_ids)}")
    print(f"  Containers : {len(containers)}")
    am = sum(c['size'] for c in containers if c['temperature'] == 'AM')
    re = sum(c['size'] for c in containers if c['temperature'] == 'RE')
    print(f"  Load       : AM={am}  RE={re}")
    print(f"  Distance   : {round(truck['route_distance_meters']/1609.34)} miles")
    print(f"  Duration   : {round(truck['route_duration_seconds']/60)} minutes")
    print()

3 truck(s) dispatched

Truck 403dce9a...
  Source     : src-0
  Route      : dst-0 -> dst-3 -> dst-1
  Containers : 6
  Load       : AM=9  RE=6
  Distance   : 180 miles
  Duration   : 191 minutes

Truck 580694d9...
  Source     : src-0
  Route      : dst-2
  Containers : 1
  Load       : AM=5  RE=0
  Distance   : 132 miles
  Duration   : 126 minutes

Truck f4d3ce41...
  Source     : src-1
  Route      : dst-3 -> dst-1 -> dst-4 -> dst-2
  Containers : 5
  Load       : AM=6  RE=6
  Distance   : 447 miles
  Duration   : 421 minutes

